# Week 11: Code Generation Agent

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/agentic-artificial-intelligence/blob/main/L11_Domain_Applications/L11_Code_Agent.ipynb)

Building a simple code generation and testing agent.

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -q langchain-openai python-dotenv
    from google.colab import userdata
    import os
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
import re
from typing import Tuple
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Ready")

## 1. Code Generation

In [ ]:
def generate_code(task: str) -> str:
    """Generate Python code for a task."""
    prompt = f"""Write Python code for this task. Return only the code, no explanations.

Task: {task}

```python
"""
    response = llm.invoke(prompt).content
    
    # Extract code block
    if "```python" in response:
        code = response.split("```python")[1].split("```")[0]
    elif "```" in response:
        code = response.split("```")[1].split("```")[0]
    else:
        code = response
    
    return code.strip()

# Test
code = generate_code("Write a function that checks if a number is prime")
print(code)

## 2. Code Testing

In [ ]:
def generate_tests(code: str, task: str) -> str:
    """Generate test cases for code."""
    prompt = f"""Write pytest test cases for this code.

Task: {task}
Code:
```python
{code}
```

Write 3-5 test cases covering edge cases."""
    
    response = llm.invoke(prompt).content
    if "```python" in response:
        tests = response.split("```python")[1].split("```")[0]
    else:
        tests = response
    return tests.strip()

tests = generate_tests(code, "Check if a number is prime")
print(tests)

In [ ]:
def run_code_safely(code: str) -> Tuple[bool, str]:
    """Execute code in a restricted environment."""
    try:
        # Create isolated namespace
        namespace = {"__builtins__": __builtins__}
        exec(code, namespace)
        return True, "Code executed successfully"
    except Exception as e:
        return False, str(e)

# Test execution
success, message = run_code_safely(code)
print(f"Execution: {'SUCCESS' if success else 'FAILED'}")
print(f"Message: {message}")

## 3. Code Agent Pipeline

In [ ]:
def code_agent(task: str, max_retries: int = 3) -> dict:
    """Complete code agent pipeline with retries."""
    result = {
        "task": task,
        "code": None,
        "tests": None,
        "success": False,
        "iterations": 0
    }
    
    for i in range(max_retries):
        result["iterations"] = i + 1
        print(f"\nIteration {i+1}/{max_retries}")
        
        # Generate code
        print("  Generating code...")
        code = generate_code(task)
        result["code"] = code
        
        # Test execution
        print("  Testing code...")
        success, error = run_code_safely(code)
        
        if success:
            print("  Code executed successfully!")
            result["tests"] = generate_tests(code, task)
            result["success"] = True
            break
        else:
            print(f"  Error: {error}")
            # Refine task with error context
            task = f"{task}\n\nPrevious attempt failed with: {error}\nPlease fix the issue."
    
    return result

# Test the agent
result = code_agent("Write a function to calculate factorial of n")
print(f"\n=== Result ===")
print(f"Success: {result['success']}")
print(f"Iterations: {result['iterations']}")
print(f"Code:\n{result['code']}")

## Summary

Built a simple code agent with:
- **Generation**: LLM-based code synthesis
- **Testing**: Automatic test generation
- **Iteration**: Retry with error feedback